In [ ]:
# Importation des modules

import pandas as pd
from gensim.models import Word2Vec
import random
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import datetime

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

In [ ]:
w2v_model_all = Word2Vec.load('results/word2vec_all_300.model')

In [ ]:
df0 = pd.read_csv("data/data_cleaned_NLP.csv", sep = ',', encoding = 'latin-1')

In [ ]:
df = df0[['Date', 'Groupe', 'Orateur', 'tokenized_replique']].copy()

In [ ]:
df['days'] = df.apply(lambda row : (datetime.date(int(row.Date[:4]),int(row.Date[5:7]),int(row.Date[8:])) - 
                    datetime.date(2017,6,28)).days, axis = 1)

In [ ]:
def add_groupe(L, etiquette):
    return [i + etiquette for i in L]

In [ ]:
def calcule_WMD_intervalle(d1, d2):
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Groupe'] == 'Novice']
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].sample(50).dropna()
        df_Exp = df1[df1['Groupe'] == 'Exp'].sample(50).dropna()
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        print(value)
        if str(value) == 'inf':
            print(i)
        WMD.append(value)
    
    px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    
    return np.mean(WMD)

In [ ]:
#calcule_WMD_intervalle(322, 336)

In [ ]:
#L = [calcule_WMD_intervalle(i, i + 14) for i in range(0, 1094, 14)]
#px.line(y = L, x = [i for i in range(0, 1094, 14)])

values = list((pd.read_csv('results/WMDinter_temporel.csv')['0']))
dates = list((pd.read_csv('results/WMDinter_temporel.csv')['Unnamed: 0']))

In [ ]:
#pd.DataFrame(L, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel0.csv')

In [ ]:
px.line(y = values, x = dates)

In [ ]:
date_label = [datetime.timedelta(days=i) + datetime.date(2017,6,28) for i in range(14, 1094, 14)]

In [ ]:
px.line(y = [np.mean(values[:i]) for i in range(1,len(values))], x = date_label, 
        labels = {'x' : 'Dates', 'y' : 'Moyenne cumulée des distances'})

On observe la moyenne cumulée des distances en fonction du temps. On constate une moyenne forte en début d'exercice, puis une relaxation pour atteindre une valeur limite.
On peut interpréter cela comme : 
- un début d'exercice où les novices et les expérimentés parlent différemment, la distance entre les deux langages est raltivement grande.
- une période d'apprentissage (du 20 septembre 2017 au 16 mai 2018) où les différences entre novices et expérimentés tendent à s'estomper (mais dans quel sens ? on s'attend à ce que les novices s'adaptent aux expérimentés, et pas forcement l'inverse), la distance diminue.
- une période de différenciation (du 16 mai 2018 au 12 décembre 2018), où la distance augmente légerement : comme si un des deux camps voulait se différencier de l'autre.
- une période stable (du 12 décembre 2018 à la fin de l'exercice).

## Analyse personnalisée député par député

In [ ]:
# Députés Novices parlant le plus (classement par nombre d'intervention)

dict(df0[df0['Groupe'] == 'Novice'].Orateur.value_counts())

In [ ]:
orateurs_novices = list(dict(df0[df0['Groupe'] == 'Novice'].Orateur.value_counts()).keys())

In [ ]:
def calcule_WMD_intervalle_personnalise(orateur, d1, d2):
    print(orateur)
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Orateur'] == orateur]
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna().sample(50)
        df_Exp = df1[df1['Groupe'] == 'Exp'].dropna().sample(50)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    
    for i in phrases_Novice:
        if len(i)<3:
            continue
        values = []
        for j in phrases_Exp:
            d = w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp'))
            if str(d) != 'inf':
                values.append(d)
        value = np.mean(values)
        #print(value)
        WMD.append(value)
    
    #px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    print(np.mean(WMD))
    
    return np.mean(WMD)

In [ ]:
L_pietraszewski = [calcule_WMD_intervalle_personnalise('laurent pietraszewski', i, i + 14) for i in range(0, 1094, 14)]
pd.DataFrame(L_pietraszewski, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_pietraszewski.csv')

### 1) Pietraszewski

In [ ]:
#L_pietraszewski = [calcule_WMD_intervalle_personnalise('laurent pietraszewski', i, i + 14) for i in range(0, 1094, 14)]
#pd.DataFrame(L_pietraszewski, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_pietraszewski.csv')

In [ ]:
px.line([np.mean(L_pietraszewski[:i]) for i in range(1,len(L_pietraszewski))])

### 2) Ruffin

In [ ]:
#L_ruffin = [calcule_WMD_intervalle_personnalise('francois ruffin', i, i + 14) for i in range(0, 1094, 14)]
#pd.DataFrame(L_ruffin, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_ruffin.csv')

In [ ]:
px.line([np.mean(L_ruffin[:i]) for i in range(1,len(L_ruffin))])

In [ ]:
for i in range(len(L_ruffin)):
    if str(L_ruffin[i]) == 'nan':
        L_ruffin[i] = L_ruffin[i-1]
    elif str(L_ruffin[i]) == 'inf':
        print(i, [(j, j+14) for j in range(0, 1094, 14)][i])

In [ ]:
L_ruffin

#### Autres députés

In [ ]:
orateurs_novices

In [ ]:
for orateur in orateurs_novices[2:11]:
    T = [calcule_WMD_intervalle_personnalise(orateur, i, i + 14) for i in range(0, 1094, 14)]
    pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + orateur + '.csv')


In [ ]:
for orateur in orateurs_novices[11:]:
    T = [calcule_WMD_intervalle_personnalise(orateur, i, i + 14) for i in range(0, 1094, 14)]
    pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + orateur + '.csv')

v

### En particulier pour les expérimentés

In [ ]:
# Députés Novices parlant le plus (classement par nombre d'intervention)

dict(df0[df0['Groupe'] == 'Exp'].Orateur.value_counts())

In [ ]:
orateurs_exp = list(dict(df0[df0['Groupe'] == 'Exp'].Orateur.value_counts()).keys())

In [ ]:
def calcule_WMD_intervalle_personnalise2(orateur, d1, d2):
    print(orateur)
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Exp = df1[df1['Orateur'] == orateur]
    df_Novice = df1[df1['Groupe'] == 'Novice']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna().sample(50)
        df_Exp = df1[df1['Groupe'] == 'Exp'].dropna().sample(50)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    
    for i in phrases_Novice:
        if len(i)<3:
            continue
        values = []
        for j in phrases_Exp:
            d = w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp'))
            if str(d) != 'inf':
                values.append(d)
        value = np.mean(values)
        #print(value)
        WMD.append(value)
    
    #px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    print(np.mean(WMD))
    
    return np.mean(WMD)

In [ ]:

T = [calcule_WMD_intervalle_personnalise2('adrien quatennens', i, i + 14) for i in range(0, 1094, 14)]
pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + 'quatennens' + '_exp.csv')





In [ ]:
for orateur in orateurs_exp:
    T = [calcule_WMD_intervalle_personnalise2(orateur, i, i + 14) for i in range(0, 1094, 14)]
    pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + orateur + '_exp.csv')



## Nombre de mots utilisés selon la période

In [ ]:
def calcule_nb_mots_periode(d1, d2):
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Orateur'] == 'Novice']
    
    
    
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna().sample(50)
        df_Exp = df1[df1['Groupe'] == 'Exp'].dropna().sample(50)
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        print(value)
        if str(value) == 'inf':
            print(i)
        WMD.append(value)
    
    px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    
    return np.mean(WMD)